In [2]:
!apt-get install default-jre
!java -version
!pip install h2o


Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
     |████████████████████████████████| 164.6MB 76kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=af0dda799a5dbff9da6a99761eea9ea59a19eeff015afa1189d197b3c1cf6b02
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


In [3]:
import pandas as pd
import numpy as np 
import spacy 

from spacy.lang.en.stop_words import STOP_WORDS as stopwords
import re
from textblob import TextBlob

from wordcloud import WordCloud 
import matplotlib.pyplot as plt 
%matplotlib inline

import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

import h2o
from h2o.automl import H2OAutoML
h2o.init()



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4j5d0smw
  JVM stdout: /tmp/tmp4j5d0smw/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4j5d0smw/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_unknownUser_ea4otr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Loading Data

In [5]:
data = pd.read_excel('/content/drive/MyDrive/DataSets/QLC /Data1.xlsx',nrows=50000)
data = data.dropna()
data.head()

,Unnamed: 0,OpportunityId,OpportunityStageName,ActivityDescription
0,0,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#ProformaInvoice\r\nDear Sir,\r\n\r\nThanks fo..."
1,1,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#Emailed\r\nDear Sir,\r\n\r\nPlease find revis..."
2,2,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
3,3,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
4,4,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,#Follow up\r\n\r\nCalled on landline number\r\...


Converting it to Dictionary

In [6]:
#convert to dictionary
cov_dic = {'OpportunityId': str,
           'OpportunityStageName': str,
           'ActivityDescription': str}
df= data.astype(cov_dic)
df=df.drop(columns=['Unnamed: 0'])
df.head()

,OpportunityId,OpportunityStageName,ActivityDescription
0,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#ProformaInvoice\r\nDear Sir,\r\n\r\nThanks fo..."
1,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#Emailed\r\nDear Sir,\r\n\r\nPlease find revis..."
2,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
3,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
4,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,#Follow up\r\n\r\nCalled on landline number\r\...


Upper case Count

In [7]:
df['upper_count'] = df['ActivityDescription'].apply(lambda x: len([t for t in x.split() if t.isupper()]))

In [8]:
# Lower case
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x:str(x).lower())

#removing '-' in id
df['OpportunityId'] = df['OpportunityId'].apply(lambda x: str(x).replace('-', ''))

#removing '/r' and '/n
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\r', ' '))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\r\n', ' '))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\n', ' '))

#count and remove URLS
df['URLs_count'] = df['ActivityDescription'].apply(lambda x: len(re.findall(r'(http|https|ftp|ssh|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&/~+#-])?',str(x))))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub(r'(http|https|ftp|ssh|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&/~+#-])?','',str(x)))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub('(?:\s)www[^, ]*', '', str(x)))

#remove single letters
df['ActivityDescription'] = df.ActivityDescription.str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
# remove all numbers
df['ActivityDescription'] = df.ActivityDescription.str.replace('\d+', '')

df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub(r'[^\w ]+',"",x)) #removing special characters
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: ' '.join(x.split())) #removing double spaces

#counting hashtags
df['hash_count'] = df['ActivityDescription'].apply(lambda x : re.findall(r'(#[Aa])', x))
df['hash_count'] =df['hash_count'].apply(lambda x : len(x))

#count email
df['e_count'] = df['ActivityDescription'].apply(lambda x : re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+",x))
df['e_count'] = df['e_count'].apply(lambda x:len(x))
#remove email 
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+","",str(x)))

#stop word count
df['sw_count']= df['ActivityDescription'].apply(lambda x: len([t for t in x.split() if t in stopwords]))
# removing stop words
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x : ' '.join([t for t in x.split() if t not in stopwords]))

In [9]:
df['URLs_count'].value_counts()

0    49547
2      149
1      105
4       82
3       73
7        3
8        2
Name: URLs_count, dtype: int64

In [10]:
#removing top 20 frequet words
text = ' '.join(df['ActivityDescription'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
f20 = freq_comm[:20]
f20

df['ActivityDescription']= df['ActivityDescription'].apply(lambda x : ' '.join([t for t in x.split() if t not in f20]))


In [11]:
#Label encoding target variable
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['OpportunityStageName']= label_encoder.fit_transform(df['OpportunityStageName']) 

In [12]:
#tokenization
from nltk.tokenize import sent_tokenize, word_tokenize 
#df['ActivityDescription']= df['ActivityDescription'].apply(lambda x: [word for word in word_tokenize(x)])
#word_tokenize(text)
df['ActivityDescription']= df['ActivityDescription'].apply(word_tokenize)

In [17]:
stemmer = nltk.stem.SnowballStemmer('english')
def tokenize(text):
   tokens = text
   stems = [stemmer.stem(item) for item in tokens if (item not in stopwords)]
   return stems

In [18]:
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x : tokenize(x))

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, random_state=111)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
train_vectors = vectorizer_tf.fit_transform(X_train.ActivityDescription)

In [21]:
test_vectors = vectorizer_tf.transform(X_test.ActivityDescription)

In [22]:
train_vectors

<39968x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 503665 stored elements in Compressed Sparse Row format>

In [23]:
vectorizer_tf.get_feature_names

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=0.75,
                max_features=1000, min_df=1, ngram_range=(1, 2), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x7f47a2981ef0>, use_idf=True,
                vocabulary=None)>

In [24]:
X_train.head()

,OpportunityId,OpportunityStageName,ActivityDescription,upper_count,URLs_count,hash_count,e_count,sw_count
45307,393AC61F59394E8BB75B04A25ADCBAD4,0,"[shanki, respons, drop, msg]",1,0,0,0,7
3558,04B567D3854046C18B0950E86DA6E1DD,8,"[pme, visit, visit, met, sudarshan, discuss, c...",1,0,0,0,8
6149,08344F755506469DA1EB6A25224014C3,2,"[firstcontactestablish, receiv, login, detail,...",0,0,0,0,6
12090,0F3935C85BF94798886834E07C3A8EE2,0,"[respond, call]",0,0,0,0,4
5487,07568ED723C34C5BA986CD1D04C1AB16,3,"[word, vishwamurthi, contact, aditya]",0,0,0,0,4


In [25]:
train_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
train_df=pd.concat([train_df,X_train['OpportunityStageName'].reset_index(drop=True)], axis=1)
train_df=pd.concat([train_df,X_train[['upper_count','hash_count','e_count','sw_count']].reset_index(drop=True)], axis=1)
train_df

,aaacqn,aaacqn id,aaacqn servic,abhishek,abl,accept,access,accord,account,account branch,account ifsc,account quantumlink,account run,accountsreceivableqlcin,accountsreceivableqlcin note,activ,ad,ad contact,ad slam,add,addit,addit year,address,address sandoz,address st,admin,advanc,advanc send,advic,agr,ahead,akshada,akshada gaikwad,ambassador,ambassador hotelmodel,amc,amc plan,amc pme,amc renew,amit,...,visit support,visit visit,wa,wait,want,web,websit,week,went,went meet,weâr,weâr happi,weâv,weâv help,whatsapp,wo,womum,wor,wor mumbai,word,work,work womum,write,wrote,year,year contract,year free,year rs,year year,yes,yesterday,youtub,youtub channel,yr,yrs,OpportunityStageName,upper_count,hash_count,e_count,sw_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,7
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.487448,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,1,0,0,8
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,6
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.735139,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,0,12
39964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,0,0,0
39965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2,0,0,10
39966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000

In [26]:
test_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
test_df=pd.concat([test_df,X_test['OpportunityStageName'].reset_index(drop=True)], axis=1)
test_df=pd.concat([test_df,X_test[['upper_count','hash_count','e_count','sw_count']].reset_index(drop=True)], axis=1)
test_df

,aaacqn,aaacqn id,aaacqn servic,abhishek,abl,accept,access,accord,account,account branch,account ifsc,account quantumlink,account run,accountsreceivableqlcin,accountsreceivableqlcin note,activ,ad,ad contact,ad slam,add,addit,addit year,address,address sandoz,address st,admin,advanc,advanc send,advic,agr,ahead,akshada,akshada gaikwad,ambassador,ambassador hotelmodel,amc,amc plan,amc pme,amc renew,amit,...,visit support,visit visit,wa,wait,want,web,websit,week,went,went meet,weâr,weâr happi,weâv,weâv help,whatsapp,wo,womum,wor,wor mumbai,word,work,work womum,write,wrote,year,year contract,year free,year rs,year year,yes,yesterday,youtub,youtub channel,yr,yrs,OpportunityStageName,upper_count,hash_count,e_count,sw_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,9,0,0,0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360888,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,9,2,0,0,6
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.430428,0.0,0.0,0.0,0.0,0.0,3,3,0,0,21
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
9989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087519,0.0,0.126784,0.0,0.10757,0.094654,0.095505,0.0,0.0,0.228195,0.10668,0.0,0.0,0.0,...,0.104722,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.082419,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3,5,0,0,39
9990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000

## Implementing H2o for model selection

In [27]:
h2o_train_df = h2o.H2OFrame(train_df)
h2o_test_df = h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
h2o_train_df.types

{'aaacqn': 'real',
 'aaacqn id': 'real',
 'aaacqn servic': 'real',
 'abhishek': 'real',
 'abl': 'real',
 'accept': 'real',
 'access': 'real',
 'accord': 'real',
 'account': 'real',
 'account branch': 'real',
 'account ifsc': 'real',
 'account quantumlink': 'real',
 'account run': 'real',
 'accountsreceivableqlcin': 'real',
 'accountsreceivableqlcin note': 'real',
 'activ': 'real',
 'ad': 'real',
 'ad contact': 'real',
 'ad slam': 'real',
 'add': 'real',
 'addit': 'real',
 'addit year': 'real',
 'address': 'real',
 'address sandoz': 'real',
 'address st': 'real',
 'admin': 'real',
 'advanc': 'real',
 'advanc send': 'real',
 'advic': 'real',
 'agr': 'real',
 'ahead': 'real',
 'akshada': 'real',
 'akshada gaikwad': 'real',
 'ambassador': 'real',
 'ambassador hotelmodel': 'real',
 'amc': 'real',
 'amc plan': 'real',
 'amc pme': 'real',
 'amc renew': 'real',
 'amit': 'real',
 'andheri': 'real',
 'andheri east': 'real',
 'annual': 'real',
 'annual mainten': 'real',
 'answer': 'real',
 'antis

In [29]:
h2o_train_df['OpportunityStageName'] = h2o_train_df['OpportunityStageName'].asfactor()
h2o_test_df['OpportunityStageName'] = h2o_test_df['OpportunityStageName'].asfactor()

In [30]:
aml = H2OAutoML(max_models = 5, seed = 10, exclude_algos = ["StackedEnsemble"], verbosity="info", nfolds=0, balance_classes=True, max_after_balance_size=0.3)

here i have set it to use top 10 models



In [31]:
x=vectorizer_tf.get_feature_names()
y='OpportunityStageName'

In [32]:
aml.train(x = x, y = y, training_frame = h2o_train_df, validation_frame=h2o_test_df)

AutoML progress: |
10:13:11.846: Project: AutoML_20210225_101311843
10:13:11.847: Cross-validation disabled by user: no fold column nor nfolds > 1.
10:13:11.853: Setting stopping tolerance adaptively based on the training frame: 0.005002001200800561
10:13:11.853: Build control seed: 10
10:13:11.853: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
10:13:15.588: training frame: Frame key: automl_training_py_1_sid_83bb    cols: 1005    rows: 35899  chunks: 41    size: 8747517  checksum: -9041490233139546984
10:13:15.591: validation frame: Frame key: py_2_sid_83bb    cols: 1005    rows: 9993  chunks: 11    size: 2370269  checksum: -5356998711068300942
10:13:15.902: leaderboard frame: Frame key: automl_leaderboard_py_1_sid_83bb    cols: 1005    rows: 4069  chunks: 41    size: 3849723  checksum: 1847102335573993059
10:13:15.902: blending frame: NULL
10:13:15.902: response 

Works best with XGBoost 

In [52]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
XGBoost_1_AutoML_20210225_101311,0.772004,0.924413,0.542064,0.293833,nan,nan
XGBoost_2_AutoML_20210225_101311,0.785972,0.921407,0.537839,0.289271,nan,nan
XGBoost_3_AutoML_20210225_101311,0.797357,0.972964,0.56577,0.320095,nan,nan
GLM_1_AutoML_20210225_101311,0.818008,1.02997,0.576233,0.332045,nan,nan
GBM_5_AutoML_20210225_101311,0.831116,1.138,0.607034,0.368491,nan,nan
GBM_4_AutoML_20210225_101311,0.83743,1.1167,0.605952,0.367178,nan,nan
GBM_3_AutoML_20210225_101311,0.838125,1.11248,0.606861,0.36828,nan,nan
GBM_2_AutoML_20210225_101311,0.839419,1.12678,0.609873,0.371945,nan,nan
GBM_1_AutoML_20210225_101311,0.84529,1.13214,0.615418,0.378739,nan,nan
DRF_1_AutoML_20210225_101311,0.856849,1.12267,0.615578,0.378936,nan,nan


In [34]:
pred=aml.leader.predict(h2o_test_df)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [35]:
aml.leader.model_performance(h2o_test_df)


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.28112500333670165
RMSE: 0.5302122248088039
LogLoss: 0.8883926391907868
Mean Per-Class Error: 0.74420768945227
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,Error,Rate
0,4821.0,2.0,15.0,360.0,0.0,0.0,0.0,0.0,63.0,94.0,0.0,0.099720,"534 / 5,355"
1,21.0,10.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.729730,27 / 37
2,210.0,0.0,50.0,53.0,0.0,0.0,0.0,0.0,2.0,22.0,0.0,0.851632,287 / 337
3,900.0,0.0,3.0,1400.0,0.0,0.0,0.0,0.0,36.0,17.0,0.0,0.405772,"956 / 2,356"
4,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,22 / 22
5,25.0,0.0,1.0,7.0,0.0,7.0,0.0,0.0,0.0,2.0,0.0,0.833333,35 / 42
6,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,5 / 5
7,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,11 / 11
8,430.0,0.0,2.0,93.0,0.0,0.0,0.0,0.0,449.0,6.0,0.0,0.541837,531 / 980
9,565.0,0.0,3.0,30.0,0.0,0.0,0.0,0.0,14.0,233.0,0.0,0.724260,612 / 845



Top-10 Hit Ratios: 


,k,hit_ratio
0,1,0.697488
1,2,0.870009
2,3,0.939157
3,4,0.973181
4,5,0.991794
5,6,0.996598
6,7,0.998499
7,8,0.999199
8,9,0.999700
9,10,1.000000


In [36]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
out = h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])
out

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20210225_101311


Model Summary: 


,,number_of_trees
0,,185.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.22351368834280275
RMSE: 0.4727723430392293
LogLoss: 0.6908092260604822
Mean Per-Class Error: 0.6708777929266566
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,Error,Rate
0,18046.0,2.0,15.0,687.0,0.0,1.0,0.0,0.0,137.0,175.0,0.0,0.053349,"1,017 / 19,063"
1,81.0,41.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.679688,87 / 128
2,719.0,0.0,425.0,103.0,0.0,0.0,0.0,0.0,9.0,50.0,0.0,0.674579,"881 / 1,306"
3,2488.0,0.0,5.0,6021.0,0.0,1.0,0.0,0.0,90.0,59.0,0.0,0.305055,"2,643 / 8,664"
4,59.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,2.0,1.0,0.0,0.956522,66 / 69
5,78.0,0.0,2.0,8.0,0.0,51.0,0.0,0.0,5.0,8.0,0.0,0.664474,101 / 152
6,17.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,20 / 20
7,30.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,33 / 33
8,1258.0,0.0,1.0,202.0,0.0,0.0,0.0,0.0,1824.0,42.0,0.0,0.451758,"1,503 / 3,327"
9,1738.0,0.0,2.0,81.0,0.0,0.0,0.0,0.0,33.0,1266.0,0.0,0.594231,"1,854 / 3,120"



Top-10 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2021-02-25 10:13:16,0.465 sec,0.0,0.909091,2.397895,0.758656,NaN,NaN,0.909091,2.397895,0.764235,NaN,NaN
1,,2021-02-25 10:13:23,7.445 sec,5.0,0.686003,1.297807,0.373409,NaN,NaN,0.687623,1.307184,0.374162,NaN,NaN
2,,2021-02-25 10:13:26,10.576 sec,10.0,0.616438,1.099369,0.359676,NaN,NaN,0.621446,1.123603,0.364955,NaN,NaN
3,,2021-02-25 10:13:29,13.489 sec,15.0,0.588504,1.018769,0.349118,NaN,NaN,0.596397,1.054464,0.357750,NaN,NaN
4,,2021-02-25 10:13:32,16.482 sec,20.0,0.573923,0.975567,0.340065,NaN,NaN,0.584808,1.021510,0.352447,NaN,NaN
5,,2021-02-25 10:13:35,19.389 sec,25.0,0.564115,0.947145,0.331987,NaN,NaN,0.577321,1.002001,0.346543,NaN,NaN
6,,2021-02-25 10:13:39,23.183 sec,30.0,0.556967,0.926358,0.326026,NaN,NaN,0.572451,0.988996,0.343040,NaN,NaN
7,,2021-02-25 10:13:42,26.674 sec,35.0,0.551426,0.909120,0.318923,NaN,NaN,0.568951,0.978447,0.339838,NaN,NaN
8,,2021-02-25 10:13:46,30.180 sec,40.0,0.546378,0.893643,0.312544,NaN,NaN,0.565998,0.970002,0.335835,NaN,NaN
9,,2021-02-25 10:13:50,34.280 sec,45.0,0.541665,0.879674,0.308031,NaN,NaN,0.563271,0.962830,0.334434,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,invoic,2625.674805,1.000000,0.034909
1,notrespond,1733.047974,0.660039,0.023042
2,pme,1699.460327,0.647247,0.022595
3,follow,1128.854980,0.429929,0.015009
4,fscall,1029.074097,0.391927,0.013682
5,spoke,952.276978,0.362679,0.012661
6,discuss,837.951172,0.319137,0.011141
7,confirm,775.071777,0.295190,0.010305
8,requir,774.411377,0.294938,0.010296
9,told,760.223145,0.289534,0.010107



See the whole table with table.as_data_frame()


In [37]:
out.convert_H2OXGBoostParams_2_XGBoostParams()

({'alpha': 0.0,
  'booster': 'gbtree',
  'colsample_bylevel': 0.8,
  'colsample_bytree': 0.8,
  'eta': 0.3,
  'gamma': 0.0,
  'gpu_id': 0,
  'grow_policy': 'depthwise',
  'lambda': 1.0,
  'max_bin': 256,
  'max_delta_step': 0.0,
  'max_depth': 10,
  'min_child_weight': 5.0,
  'nround': 10000,
  'nthread': 2,
  'num_class': 11,
  'objective': 'multi:softprob',
  'seed': 10,
  'silent': True,
  'subsample': 0.6,
  'updater': 'grow_gpu_hist'},
 10000)

In [38]:

xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, objective='multi:softprob', random_state=10, **{"updater": "grow_gpu"})

In [39]:

xgb_clf.fit(train_vectors, X_train['OpportunityStageName'])

[11:16:35] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, updater='grow_gpu', verbosity=1)

In [ ]:
predictions = xgb_clf.predict(test_vectors)

In [ ]:
cm = confusion_matrix(X_test['OpportunityStageName'], predictions)
print(cm)

In [ ]:
print('classification_report :\n',classification_report(X_test['OpportunityStageName'], predictions))

## Class Imbalance

In [54]:
X_train['OpportunityStageName'].value_counts()

0     21166
3      9669
8      3726
9      3475
2      1466
5       170
1       143
4        74
7        38
6        23
10       18
Name: OpportunityStageName, dtype: int64

In [43]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(X_train['OpportunityStageName']),
                                             X_train['OpportunityStageName']))

In [ ]:
class_weights

In [45]:
weights = np.ones(X_train.shape[0], dtype = 'float')
weights

array([1., 1., 1., ..., 1., 1., 1.])

In [46]:
for i, val in enumerate(X_train['OpportunityStageName']):
    weights[i] = class_weights[val]

weights

array([0.17166468, 0.97516225, 2.47848195, ..., 0.17166468, 0.3757839 ,
       0.17166468])

In [47]:
xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, objective='multi:softprob', random_state=10, **{"updater": "grow_gpu"})

In [48]:
xgb_clf.fit(train_vectors, X_train['OpportunityStageName'], sample_weight=weights)

[11:16:45] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, updater='grow_gpu', verbosity=1)

In [ ]:
predictions = xgb_clf.predict(test_vectors)

In [ ]:
cm = confusion_matrix(X_test['OpportunityStageName'], predictions)
print(cm)

In [51]:
print('classification_report :\n',classification_report(X_test['OpportunityStageName'], predictions))

classification_report :
               precision    recall  f1-score   support

           0       0.75      0.32      0.45      5355
           1       0.04      0.68      0.08        37
           2       0.12      0.44      0.19       337
           3       0.61      0.52      0.56      2356
           4       0.02      0.50      0.03        22
           5       0.10      0.55      0.17        42
           6       0.01      1.00      0.03         5
           7       0.01      0.18      0.02        11
           8       0.42      0.49      0.45       980
           9       0.30      0.42      0.35       845
          10       0.01      0.33      0.01         3

    accuracy                           0.40      9993
   macro avg       0.22      0.49      0.21      9993
weighted avg       0.61      0.40      0.45      9993

classification_report :
               precision    recall  f1-score   support

           0       0.75      0.32      0.45      5355
           1       0.04     